Must needed setup before every assignment

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 215.7MB 57kB/s 
     |████████████████████████████████| 204kB 38.2MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=582c7f54de92cd29e64c49cc962040b0c569cd34b3e5b4b6340830fc1c6af3dc
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u232-b09-0ubuntu1~18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Imports

In [0]:
import itertools
from pyspark import SparkContext, SparkConf
from pyspark.sql import *
from pyspark.sql.functions import *


Setup Spark

In [0]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [0]:
Code Variables

In [0]:
s        = 100 # set support threshold
topN     = 20  # set top items to show
fileName = 'browsing.txt'

Code Functions

In [0]:
def single_and_pairs(line):
  items  = line.split()
  single = [ ( ( item, 'Total'), 1 )  for item in items ]
  pairs  = [ ( pair, 1 ) for pair in itertools.permutations(items, 2)]
  return single + pairs

def sort_filter(counts): #and put the total in the end
  total, results = 0, sorted(list(counts), key=lambda (key, count): -count)
  for key, count in results:
    if key == 'Total':
      total = count
  if total > s:
    results.remove(('Total', total))
    results.append(('Total', total))
    return results #so Total will be always at the end.

def calcCS(pair):
  k1, counts, total = pair[0], pair[1], 0
  if counts != None:
    total = float( counts.pop(-1)[1] )
    return [ ((k1, k2), count/total) for k2, count in counts ]


Output

In [0]:
confidenceRDD = (sc.textFile(fileName, 12 ) #partition goes here
                  .flatMap( single_and_pairs )
                  .reduceByKey(lambda a, b: a + b)
                  .map(lambda ((key, other), count):(key, (other, count)))
                  .groupByKey()
                  .mapValues( sort_filter )
                  .filter(lambda (key, values): values != None)
                  .flatMap( calcCS )
                  )

print "Top " + str(topN)
#print confidenceRDD.collect()
print confidenceRDD.takeOrdered(topN, lambda (itemSet, score): -score)

Top 20
[((u'DAI93865', u'FRO40251'), 1.0), ((u'GRO85051', u'FRO40251'), 0.999176276771005), ((u'GRO38636', u'FRO40251'), 0.9906542056074766), ((u'ELE12951', u'FRO40251'), 0.9905660377358491), ((u'DAI88079', u'FRO40251'), 0.9867256637168141), ((u'FRO92469', u'FRO40251'), 0.983510011778563), ((u'DAI43868', u'SNA82528'), 0.972972972972973), ((u'DAI23334', u'DAI62779'), 0.9545454545454546), ((u'DAI74977', u'GRO83463'), 0.8240740740740741), ((u'DAI20027', u'DAI70456'), 0.8070175438596491), ((u'DAI33885', u'GRO46854'), 0.7407407407407407), ((u'SNA46500', u'GRO44993'), 0.7403846153846154), ((u'SNA81556', u'DAI85309'), 0.7363636363636363), ((u'ELE92920', u'DAI62779'), 0.7326649958228906), ((u'DAI53152', u'FRO40251'), 0.717948717948718), ((u'SNA18336', u'DAI62779'), 0.7136812411847673), ((u'ELE55848', u'GRO32086'), 0.7094594594594594), ((u'DAI74112', u'DAI42493'), 0.6991150442477876), ((u'GRO89004', u'ELE25077'), 0.698051948051948), ((u'GRO81647', u'GRO73461'), 0.6775510204081633)]
